In [1]:
!pip install -Uqq lightning segmentation-models-pytorch

In [2]:
import numpy as np
import torch
from glob import glob
import lightning.pytorch as pl
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import albumentations as A
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from torch import nn
import segmentation_models_pytorch as smp
import cv2
import torch.nn.functional as F
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.notebook import tqdm

from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, GradientAccumulationScheduler, LearningRateMonitor

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [4]:
def get_cancer_status(images):
    masks = [image.replace("FULL", "MASK") for image in images]
    cancer_status = np.array([np.isin(2, Image.open(mask)).any() for mask in masks])
    return cancer_status.astype(int)

def splitter(images, splits, random_state=0):
    images = sorted(images)
    
    patients = np.array([str(x).split("/")[-2] for x in images]) 
    cancers = get_cancer_status(images)
    
    gkf = StratifiedGroupKFold(n_splits=splits, shuffle=True, random_state=random_state)
    return list(gkf.split(X=images, y=cancers, groups=patients))

In [5]:
def diff_lr(model, lr, weight_decay=0, diff=100, no_decay=["bias", "norm"], body=["embeddings", "encoder"]):
    
    head_params = [(n,p) for n, p in model.named_parameters() if not any(body_param in n for body_param in body)]
    body_params = [(n,p) for n, p in model.named_parameters() if any(body_param in n for body_param in body)]
    
    #head lr
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in head_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": lr,
        },
        {
            "params": [p for n, p in head_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": lr,
        },
    ]

    # body lr
    optimizer_grouped_parameters += [
        {
            "params": [p for n, p in body_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": lr/diff,
        },
        {
            "params": [p for n, p in body_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": lr/diff,
        },
    ]

    return optimizer_grouped_parameters


In [6]:
class MammoDataset(Dataset):
    def __init__(self, images, transforms=None):
        self.images = images
        self.transforms = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        mask_path = self.images[idx].replace("FULL", "MASK")
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)

        if self.transforms:
            transformed = self.transforms(image=np.array(image), mask=np.array(mask).astype(np.uint8))
            image = transformed['image']
            mask = transformed['mask']

        #Use the feature extractor
        feature_extractor = SegformerImageProcessor(do_reduce_labels=False, 
                                                    do_resize=False,
                                                    do_normalize=True,)
        encoded_inputs = feature_extractor(images=image, segmentation_maps=mask, return_tensors="pt")

        for k,_ in encoded_inputs.items():
            encoded_inputs[k].squeeze_()

        return encoded_inputs

In [7]:
class MammoDataModule(pl.LightningDataModule):
    def __init__(self, 
                 train_image_dir: str, test_image_dir: str, 
                 batch_size: int = 8, 
                 train_transforms=None, val_transforms=None,
                 splits = 4, fold = 0, num_workers=4, random_state=123):
        super().__init__()
        self.train_image_dir = train_image_dir
        self.test_image_dir = test_image_dir
        self.batch_size = batch_size
        self.train_transforms = train_transforms
        self.val_transforms = val_transforms
        self.num_workers = num_workers
        self.fold = fold
        self.splits = splits
        self.random_state = random_state

    def setup(self, stage):
        all_images = np.array(self._get_all_images(self.train_image_dir))
        splits = splitter(all_images, self.splits, self.random_state)
        train_idx, val_idx = splits[self.fold]
        train_images, val_images = all_images[train_idx], all_images[val_idx]

        self.train_dataset = MammoDataset(train_images, self.train_transforms)
        self.val_dataset = MammoDataset(val_images, self.val_transforms)

        test_images = self._get_all_images(self.test_image_dir)
        self.test_dataset = MammoDataset(test_images, self.val_transforms)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=True)
    
    def _get_all_images(self, image_dir: str):
        return glob(f"{image_dir}/**/*_FULL.png", recursive=True)

In [8]:
class SegformerFinetuner(pl.LightningModule):
    
    def __init__(self, model: str, loss: nn.Module, id2label: dict, lr: float, wd: float):
        super().__init__()
        
        self.id2label = id2label
        self.num_classes = len(id2label.keys())
        self.label2id = {v:k for k,v in self.id2label.items()}
        
        self.model = SegformerForSemanticSegmentation.from_pretrained(
            model, 
            return_dict=False, 
            num_labels=self.num_classes,
            id2label=self.id2label,
            label2id=self.label2id,
        )
        self.criterion = loss
        self.lr = lr
        self.weight_decay = wd
        
        self.metrics = {}
        self.validation_step_outputs = []
        self.validation_step_labels = []
        
    def forward(self, images):
        outputs = self.model(pixel_values=images)

        upsampled_logits = self._upsample_logits(outputs[0], images)
        
        return(upsampled_logits)
    
    def training_step(self, batch, batch_nb):
        
        loss, _, _ = self._calculate_loss(batch)
        
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_nb):
        
        loss, upsampled_logits, masks = self._calculate_loss(batch)
        
        predicted = upsampled_logits.argmax(dim=1)
        
        self.validation_step_outputs.append(predicted.detach().cpu().long())
        self.validation_step_labels.append(masks.detach().cpu())
        
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        
        return loss
    
    def on_validation_epoch_end(self):
        
        all_preds = torch.cat(self.validation_step_outputs)
        all_labels = torch.cat(self.validation_step_labels)
        
        tp, fp, fn, tn = smp.metrics.get_stats(all_preds, all_labels, mode='multiclass', num_classes=3)
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro")
        accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
        f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction="macro")
        
        metrics = {"val_mean_iou":iou_score, "val_mean_accuracy":accuracy, "val_mean_f1_score": f1_score}
        self.log_dict(metrics, prog_bar=True)
        self.metrics = metrics
        
        self.validation_step_outputs.clear()
        self.validation_step_labels.clear()
    
    
    def configure_optimizers(self):
        grouped_params = diff_lr(self.model, lr = self.lr, weight_decay = self.weight_decay)
        optimizer = torch.optim.AdamW(grouped_params)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3,factor=0.1, min_lr=1e-8, threshold=0.01, verbose=True),
                "monitor": "val_loss"
            },
        }
    
    def _upsample_logits(self, logits, masks):
        return nn.functional.interpolate(
            logits, 
            size=masks.shape[-2:], 
            mode="bilinear", 
            align_corners=False
        )
    
    def _calculate_loss(self, batch):
        images, masks = batch['pixel_values'], batch['labels']
        
        outputs = self.model(pixel_values=images, labels=masks)
        
        logits = outputs[1]
        
        upsampled_logits = self._upsample_logits(logits, masks)

        loss = self.criterion(upsampled_logits, masks)

        return loss, upsampled_logits, masks


In [9]:
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    #A.RandomRotate90(p=0.5),
    #A.Transpose(p=0.5),
    A.RandomBrightnessContrast(p=0.5, brightness_limit=0.05, contrast_limit=0.05),
])


In [10]:
TRAIN_DIR = "/kaggle/input/diplomski-datapreprocessing-512xalleq/data/train/processed_512x_alleq"
TEST_DIR = "/kaggle/input/diplomski-datapreprocessing-512xalleq/data/test/processed_512x_alleq"

id2label = {0:"BACKGROUND", 1:"BENIGN", 2:"MALIGNANT"}
NUM_FOLDS = 4

all_metrics = {"val_mean_iou": [], "val_mean_accuracy": [], "val_mean_f1_score": []}

In [11]:
class ComboLoss(nn.Module):
    
    def __init__(self, alpha, w1=0.5, w2=0.5, smooth=0):
        super().__init__()
        self.w1 = w1
        self.w2 = w2
        self.multi_criterion = smp.losses.DiceLoss(mode="multiclass", smooth=smooth)
        self.binary_criterion = smp.losses.TverskyLoss(mode="binary", alpha = alpha, beta = 1 - alpha)
    
    def forward(self, output, target):
        multi_loss = self.multi_criterion(output, target)
        
        binary_output = nn.functional.softmax(output, dim=1)[:, 1:, :, :].sum(dim=1, keepdim=True)
        binary_target = (target != 0).long()
        binary_loss = self.binary_criterion(binary_output, binary_target)
    
        return self.w1 * multi_loss + self.w2 * binary_loss

In [12]:
MODEL_FOLDER = "/kaggle/input/diplomski-training/models"

In [14]:
dm = MammoDataModule(
        TRAIN_DIR, 
        TEST_DIR, 
        batch_size=32,
        train_transforms=train_transforms,
        num_workers=2,
        splits=4,
        fold=0,
        random_state=123)

In [ ]:
# Initialize a tensor to store the sum of all predictions
sum_preds = None
last_masks = None

dm.setup(stage="test")
for k in range(NUM_FOLDS):
   
    MODEL_NAME = f"segformer-fold-{k}"
    segformer_finetuner = SegformerFinetuner(
    f"./models/{MODEL_NAME}",
    nn.CrossEntropyLoss(),
    id2label,
    lr=1e-3,
    wd=0)
       
    fold_preds = []
    fold_masks = []
    
    segformer_finetuner.eval()
    
    with torch.no_grad():
        for batch in tqdm(dm.test_dataloader()):
            images, masks = batch['pixel_values'], batch['labels']
            upsampled_logits = segformer_finetuner(images)
            
            probs = torch.nn.functional.softmax(upsampled_logits, dim=1)
            fold_preds.append(probs.detach().cpu())
            fold_masks.append(masks.detach().cpu())
    
    # Combine the predictions and masks of the fold
    fold_preds = torch.cat(fold_preds, dim=0)
    fold_masks = torch.cat(fold_masks, dim=0)

    # Add the fold predictions to the sum
    if sum_preds is None:
        sum_preds = fold_preds
    else:
        sum_preds += fold_preds

    # Save the masks from the last fold

# Average the predictions by dividing the sum by the number of folds
avg_preds = sum_preds / NUM_FOLDS

# Get the predicted classes
pred_classes = torch.argmax(avg_preds, dim=1)

tp, fp, fn, tn = smp.metrics.get_stats(pred_classes, fold_masks, mode='multiclass', num_classes=3)
iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro")
accuracy = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
f1_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction="macro")

print(f'Dice Score: {f1_score}\n'
      f'IoU Score: {iou_score}\n'
      f'Accuracy: {accuracy}')